# Welcome to this kernel

The goal of this kernel is very simple. It aims to provide some useful insights about the data and hopefully can guide you into what features to generate and how to tackle the modelling part.

<a id ="table_of_contents"></a>
# Table of contents


[Imports](#imports)

[Quick look at shops df](#quick_look_shops)

[Fix shops df and generate some features](#fix_shops)

[Quick look at item category df](#quick_look_item_cat)

[Quick look at items df](#quick_look_item)

[Quick look at sales df](#quick_look_sales)

[Joining df](#join_df)

[Exploratory Data Analysis (EDA)](#eda)

[Viz of sales per week, month of shops and item_category columns](#sales_viz)

[Total sales and the variation on secondary axis](#sales_viz_2_axis)

--> [Question 1: Create a plot with the moving average of total sales (7 days) and the variation on the second axis.](#question_1)

[Calendar heatmap](#calendar_heatmap)

[Timeseries autocorrelation and partial autocorrelation plots: daily sales](#corr_plots_daily)

[Manually calculate the Partial Autocorrelation](#autocorrelation_calculation)

[Timeseries decomposition plots: weekly sales](#decomp_weekly)

--> [Question 2: Create a decomposition plot for a city of weekly sales](#question_2)

[Visualizing the most important cities](#viz_cities)

--> [Question 3: Create a treemap plot for item_category and the total combined sales](#question_3)

[Visualizing nulls values](#viz_null_values)

[Visualization of outliers](#viz_outliers)

[Conclusion](#conclusion)

<a id = "imports"></a>
# Imports
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# make calendar maps
!pip install calmap

In [ ]:
import calmap

In [ ]:
# Main libraries that we will use in this kernel
import datetime
import numpy as np
import pandas as pd

# # garbage collector: free some memory is needed
import gc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# pip install squarify (algorithm for treemap) if missing
import squarify

# statistical package and some useful functions to analyze our timeseries
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.stattools as stattools

import time

from xgboost import XGBRegressor
from string import punctuation
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

def print_files():
    import os
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Let's see how many different files we are dealing with
print_files()

<a id = "quick_look_shops"></a>
# Quick look at shops df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# import the df
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
shops.shape

In [ ]:
shops.head()

In [ ]:
# We don't have any duplicates in the shop_name field
shops.shape[0] == len(shops["shop_name"].unique())

In [ ]:
# However inspecting the df by name, we can see that shop_id 10 and 11 are very similar. Later we will try and group them once we inspect the sales per shop
shops[shops["shop_id"].isin([10, 11])]

In [ ]:
# The same happens with the shops with shop_id 23 and 24
shops[shops["shop_id"].isin([23, 24])]

In [ ]:
# No missing values in the shops df
shops.isnull().sum().sum()

<a id = "fix_shops"></a>
# Fix shops df and generate some features
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# let's correct the shops df and also generate a few more features
def fix_shops(shops):
    '''
    This function modifies the shops df inplace.
    It correct's 3 shops that we have found to be 'duplicates'
    and also creates a few more features: extracts the city and encodes it using LabelEncoder
    '''
    
    d = {0:57, 1:58, 10:11, 23:24}
    
    # this 'tricks' allows you to map a series to a dictionary, but all values that are not in the dictionary won't be affected
    # it's handy since if we blindly map the values, the missings values will be replaced with nan
    shops["shop_id"] = shops["shop_id"].apply(lambda x: d[x] if x in d.keys() else x)
    
    # replace all the punctuation in the shop_name columns
    shops["shop_name_cleaned"] = shops["shop_name"].apply(lambda s: "".join([x for x in s if x not in punctuation]))
    
    # extract the city name
    shops["city"] = shops["shop_name_cleaned"].apply(lambda s: s.split()[0])
    
    # encode it using a simple LabelEncoder
    shops["city_id"] = LabelEncoder().fit_transform(shops['city'])

In [ ]:
# apply our function to the shops_df
fix_shops(shops)

In [ ]:
shops.head()

<a id = "quick_look_item_cat"></a>
# Quick look at items_category df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# import df
items_category = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
items_category.shape

In [ ]:
items_category.head()

In [ ]:
# We don't have any duplicates in the item_category_name field
items_category.shape[0] == len(items_category["item_category_name"].unique())

In [ ]:
# allow pandas to show all the rows from this df
pd.options.display.max_rows = items_category.shape[0]

In [ ]:
# If we take a closer look, we can see that we have a lot of Play Station categories: like accesories, games and so on. We have the same categories for XBOX and also for PC Games.
# A lot of categories have to deal with books, presents and computer software and music (CD).
# We will generate later some features by parsing the names and making groupedby features.
items_category.head()

In [ ]:
# If we apply a simple lambda function and extract the everything that contains PS, we will get 16 different categories for PlayStation
items_category["PS_flag"] = items_category["item_category_name"].apply(lambda x: True if "PS" in x else False)
items_category[items_category["PS_flag"] == True]

In [ ]:
# No missing values in the items_category df
items_category.isnull().sum().sum()

<a id = "quick_look_item"></a>
# Quick look at items df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# import df
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
items.shape

In [ ]:
# allow pandas to show all the rows from this df
pd.options.display.max_rows = items.shape[0]
items.head()

In [ ]:
# We have a lot of items_id, and as we can see some of them are very familiar.
items[items["item_id"].isin([69, 70])]["item_name"].iloc[0]

In [ ]:
items[items["item_id"].isin([69, 70])]["item_name"].iloc[1]

In [ ]:
# No missing values in the items category
items.isnull().sum().sum()

In [ ]:
# Let's see the top 10 and bottom 10 item categories
items_gb = items.groupby("item_category_id").size().to_frame()

In [ ]:
items[items["item_category_id"] == 60]

In [ ]:
items_gb.rename(columns = {0:"counts"}, inplace = True)

In [ ]:
items_gb.sort_values("counts", ascending = False, inplace = True)

In [ ]:
top_10 = items_gb[:10]

In [ ]:
bottom_10 = items_gb[-10:]

In [ ]:
top_10 = top_10.append(bottom_10)
top_10 = top_10.sort_values("counts", ascending = False)

In [ ]:
top_10.reset_index()

In [ ]:
# We can notice that in the top 10 most popular items products we have PS3
# At the same time, in the bottom 10 products, we can find 2 PS2.
# This means, that we have to be careful while generating features like PS
pd.merge(top_10, items_category, left_on = "item_category_id", right_on = "item_category_id")

<a id = "quick_look_sales"></a>
# Quick look at sales df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# import df
sales = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
sales.shape

In [ ]:
sales.sample(10)

In [ ]:
sales.info()

In [ ]:
# No null values in the sales df

# Is this True?

sales.isnull().sum()

In [ ]:
sorted(list(sales["item_cnt_day"].unique()))[:20]

In [ ]:
del shops, items_category, items, sales
gc.collect()

<a id = "join_df"></a>
# Joining df
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# a simple function that creates a global df with all joins and also shops corrections
def create_df():
    '''
    This is a helper function that creates the train df.
    '''
    # import all df
    shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
    fix_shops(shops) # fix the shops as we have seen before
    
    items_category = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
    items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
    sales = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
    
    # fix shop_id in sales so that we can leater merge the df
    d = {0:57, 1:58, 10:11, 23:24}
    sales["shop_id"] = sales["shop_id"].apply(lambda x: d[x] if x in d.keys() else x)
    
    # create df by merging the previous dataframes
    df = pd.merge(items, items_category, left_on = "item_category_id", right_on = "item_category_id")
    df = pd.merge(sales, df, left_on = "item_id", right_on = "item_id")
    df = pd.merge(df, shops, left_on = "shop_id", right_on = "shop_id")
    
    # convert to datetime and sort the values
#     df["date"] = pd.to_datetime(df["date"], format = "%d.%m.%Y")
    df.sort_values(by = ["shop_id", "date"], ascending = True, inplace = True)
    
    return df

In [ ]:
df = create_df()

In [ ]:
df.shape

In [ ]:
df.head()

<a id = "eda"></a>
# Exploratory Data Analysis (EDA)
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# It seems that there are no null values, however this is not fully true. 
# As we will see in the next section, when we groupby and plot the data, there are a lot of months where there have been no sales so basically it's a null value, and we have to impute zero sales for that month.
df.isnull().sum().sum()

<a id = "sales_viz"></a>
# Viz of sales per week, month of shops and item_category columns
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df.info()

In [ ]:
# Let's group by Month and see all the sales

# resample in timeseries is the same as groupby
# in order it to work, we must set the date column as index, and it must be a datetime format (strings are not valid)
# when we resample it, we can pass D: daily, W: weekly or M: monthly
# we can then perform operation on the 'resampled' columns like
# sum, mean and others.

# calculate the monthly sales
df["date"] = pd.to_datetime(df["date"], format = "%d.%m.%Y")

In [ ]:
df["Year"] = df["date"].dt.year

In [ ]:
df["Month"] = df["date"].dt.month

In [ ]:
df.head()

In [ ]:
# resample the data on a monthly basis
x = df[["date", "item_cnt_day"]].set_index("date").resample("M").sum()

# plot the data using matplotlib
plt.figure(figsize = (10, 6))
plt.plot(x, color = "blue", label = "Monthly sales")
plt.title("Monthly sales")
plt.legend();

From our very first and simple figure, we can already extract very useful information.
* First of all, we can see big spikes in January, like to be motivated with national holidays in Russia.
* Second: we see a general trend to decline in our timeseries. If you are planning to use a parametrical model, you must take into account this.

In [ ]:
# perform the same operations but on a weekly basis
x = df[["date", "item_cnt_day"]].set_index("date").resample("W").sum()

plt.figure(figsize = (10, 6))
plt.plot(x.index, x, color = "blue", label = "Weekly sales")
plt.title("Weekly sales")
plt.legend();

Analyzing data on a weekly basis, gives us much more information. We can see more variation between weeks, but the main point stays the same: we have spines in January and sales that go down overtime.

In the next plots we will represent the monthly sales (left plot) and weekly sales (right plot) for each shop. 

In the light red/pink areas of each plot, we will mark the national holidays in Russia and see if there is any connection with sales spikes.

In [ ]:
russian_holidays_start = [
datetime.datetime(2013, 1, 1),
datetime.datetime(2013, 2, 23),
datetime.datetime(2013, 3, 8),
datetime.datetime(2013, 5, 1),
datetime.datetime(2013, 5, 9),
datetime.datetime(2013, 6, 12),
datetime.datetime(2013, 11, 4),

datetime.datetime(2014, 1, 1),
datetime.datetime(2014, 2, 23),
datetime.datetime(2014, 3, 8),
datetime.datetime(2014, 5, 1),
datetime.datetime(2014, 5, 9),
datetime.datetime(2014, 6, 12),
datetime.datetime(2014, 11, 4),

datetime.datetime(2015, 1, 1),
datetime.datetime(2015, 2, 23),
datetime.datetime(2015, 3, 8),
datetime.datetime(2015, 5, 1),
datetime.datetime(2015, 5, 9),
datetime.datetime(2015, 6, 12),
datetime.datetime(2015, 11, 4)
]

In [ ]:
russian_holidays_end = [
datetime.datetime(2013, 1, 8),
datetime.datetime(2013, 2, 23),
datetime.datetime(2013, 3, 8),
datetime.datetime(2013, 5, 1),
datetime.datetime(2013, 5, 9),
datetime.datetime(2013, 6, 12),
datetime.datetime(2013, 11, 4),

datetime.datetime(2014, 1, 8),
datetime.datetime(2014, 2, 23),
datetime.datetime(2014, 3, 8),
datetime.datetime(2014, 5, 1),
datetime.datetime(2014, 5, 9),
datetime.datetime(2014, 6, 12),
datetime.datetime(2014, 11, 4),

datetime.datetime(2015, 1, 8),
datetime.datetime(2015, 2, 23),
datetime.datetime(2015, 3, 8),
datetime.datetime(2015, 5, 1),
datetime.datetime(2015, 5, 9),
datetime.datetime(2015, 6, 12),
datetime.datetime(2015, 11, 4)
]

In [ ]:
for iterable in sorted(list(df["shop_name"].unique())[:5]):

    # create the size of the figure
    plt.figure(figsize = (30, 10))

    # create the subplot for Monthly sales of the each shop
    plt.subplot(1, 2, 1)
    
    # calculate the Monthly sales of each shop
    short_df = df[df["shop_name"] == iterable][["date","item_cnt_day"]]
    short_df["date"] = pd.to_datetime(short_df["date"], format = "%d.%m.%Y")
    short_df["YEAR"] = short_df["date"].dt.year
    short_df = short_df.set_index("date").groupby("YEAR").resample("M")["item_cnt_day"].sum()
    short_df = short_df.reset_index()
    
    # adding moving average
    short_df["MA3M"] = short_df["item_cnt_day"].rolling(window = 3).mean()
    short_df["MA4M"] = short_df["item_cnt_day"].rolling(window = 4).mean()
    short_df["MA5M"] = short_df["item_cnt_day"].rolling(window = 5).mean()
    
    # assing the data to plot
    sales = short_df["item_cnt_day"]
    dates = short_df["date"]
    
    average_3_months = short_df["MA3M"]
    average_4_months = short_df["MA4M"]
    average_5_months = short_df["MA5M"]

    # plot the data and add label
    plt.plot(dates, sales, 'o-', label = "Monthly sales")
    
    plt.plot(dates, average_3_months, '.-', label = "Average sales of the last 3 months")

    # get current axis and plot the areas
    ax = plt.gca()
    alpha = 0.2
    
    for start_date, end_date in zip(russian_holidays_start, russian_holidays_end):
        
        # add shaded areas for holidays 2013
        ax.axvspan(start_date, end_date, alpha = alpha, color = 'red')    
       
    # add title and show legend    
    plt.title('Monthly sales of shop {}'.format(iterable))
    plt.ylabel('Total Monthly sales of shop {}'.format(iterable))
    plt.xlabel("Time grouped by month")
    plt.legend()
    
    #######################################################################################
    # Weekly sales
    #######################################################################################
    
    plt.subplot(1, 2, 2)
    
      # calculate the Weekly sales of each shop
    short_df = df[df["shop_name"] == iterable][["date","item_cnt_day"]]
    short_df["date"] = pd.to_datetime(short_df["date"], format = "%d.%m.%Y")
    short_df["YEAR"] = short_df["date"].dt.year
    short_df = short_df.set_index("date").groupby("YEAR").resample("W")["item_cnt_day"].sum()
    short_df = short_df.reset_index()
    
    # adding moving average
    short_df["MA3W"] = short_df["item_cnt_day"].rolling(window=3).mean()
    short_df["MA4W"] = short_df["item_cnt_day"].rolling(window=4).mean()
    short_df["MA5W"] = short_df["item_cnt_day"].rolling(window=5).mean()
    
    # assing the data to plot
    
    # general sales
    sales = short_df["item_cnt_day"]
    dates = short_df["date"]
    
    average_3_weeks = short_df["MA3W"]
    average_4_weeks = short_df["MA4W"]
    average_5_weeks = short_df["MA5W"]

    # plot the data and add label
    plt.plot(dates, sales, 'o-', label = "Weekly sales")
    plt.plot(dates, average_3_weeks, '.-', label = "Average sales of the last 3 weeks")
    
    # get current axis and plot the areas
    ax = plt.gca()
    
    for start_date, end_date in zip(russian_holidays_start, russian_holidays_end):
        
        # add shaded areas for holidays 2013
        ax.axvspan(start_date, end_date, alpha = alpha, color = 'red')
    
    # add title and show legend
    plt.title('Weekly sales of shop {}'.format(iterable))
    plt.ylabel('Total Weekly sales of shop {}'.format(iterable))
    plt.xlabel("Time grouped by week")
    plt.legend()
    
    # general sales
    plt.show()

In the next plots we will represent the monthly sales (left plot) and weekly sales (right plot) for each item category. 

We will also plot the percentile 5 and 95 for each shop by year.

In the light red/pink areas of each plot, we will mark the national holidays in Russia and see if there is any connection with sales spikes.

In [ ]:
for iterable in sorted(list(df["item_category_name"].unique()))[:5]:

    # create the size of the figure
    plt.figure(figsize = (30, 10))

    # create the subplot for Monthly sales of the each shop
    plt.subplot(1, 2, 1)
    
    # calculate the Monthly sales of each shop
    short_df = df[df["item_category_name"] == iterable][["date","item_cnt_day"]]
    short_df["date"] = pd.to_datetime(short_df["date"], format = "%d.%m.%Y")
    short_df["YEAR"] = short_df["date"].dt.year
    short_df = short_df.set_index("date").groupby("YEAR").resample("M")["item_cnt_day"].sum()
    short_df = short_df.reset_index()
    
    # adding moving average
    short_df["MA3M"] = short_df["item_cnt_day"].rolling(window=3).mean()
    short_df["MA4M"] = short_df["item_cnt_day"].rolling(window=4).mean()
    short_df["MA5M"] = short_df["item_cnt_day"].rolling(window=5).mean()
    
    # assing the data to plot
    sales = short_df["item_cnt_day"]
    dates = short_df["date"]
    
    average_3_months = short_df["MA3M"]
    average_4_months = short_df["MA4M"]
    average_5_months = short_df["MA5M"]

    # plot the data and add label
    plt.plot(dates, sales, 'o-', label = "Monthly sales")
    
    plt.plot(dates, average_3_months, '.-', label = "Average sales of the last 3 months")

    # get current axis and plot the areas
    ax = plt.gca()
    alpha = 0.2
    
    for start_date, end_date in zip(russian_holidays_start, russian_holidays_end):
        
        # add shaded areas for holidays 2013
        ax.axvspan(start_date, end_date, alpha = alpha, color = 'red')   
    
    # add title and show legend
    plt.title('Monthly sales of item category {}'.format(iterable))
    plt.ylabel('Total Monthly sales of item category {}'.format(iterable))
    plt.xlabel("Time grouped by month")
    plt.legend()
    

    #######################################################################################
    # Weekly sales
    #######################################################################################
    
    plt.subplot(1, 2, 2)
    
      # calculate the Weekly sales of each shop
    short_df = df[df["item_category_name"] == iterable][["date","item_cnt_day"]]
    short_df["date"] = pd.to_datetime(short_df["date"], format = "%d.%m.%Y")
    short_df["YEAR"] = short_df["date"].dt.year
    short_df = short_df.set_index("date").groupby("YEAR").resample("W")["item_cnt_day"].sum()
    short_df = short_df.reset_index()
    
    # adding moving average
    short_df["MA3W"] = short_df["item_cnt_day"].rolling(window = 3).mean()
    short_df["MA4W"] = short_df["item_cnt_day"].rolling(window = 4).mean()
    short_df["MA5W"] = short_df["item_cnt_day"].rolling(window = 5).mean()
    
    # assing the data to plot
    
    # general sales
    sales = short_df["item_cnt_day"]
    dates = short_df["date"]
    
    average_3_weeks = short_df["MA3W"]
    average_4_weeks = short_df["MA4W"]
    average_5_weeks = short_df["MA5W"]

    # plot the data and add label
    plt.plot(dates, sales, 'o-', label = "Weekly sales")
    plt.plot(dates, average_3_weeks, '.-', label = "Average sales of the last 3 weeks")
    
    # get current axis and plot the areas
    ax = plt.gca()
    
    for start_date, end_date in zip(russian_holidays_start, russian_holidays_end):
        
        # add shaded areas for holidays 2013
        ax.axvspan(start_date, end_date, alpha = alpha, color = 'red')
        
    # add title and show legend
    plt.title('Weekly sales of item category {}'.format(iterable))
    plt.ylabel('Total Weekly sales of item category {}'.format(iterable))
    plt.xlabel("Time grouped by week")
    plt.legend()
    # general sales
    plt.show()

<a id = "sales_viz_2_axis"></a>
# Total sales and the variation on secondary axis
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# we can observe a general trend of decrasing sales.
# let's add a second axis to see the variation of intradays sales

# select the columns of interest
df_var = df[["date", "item_cnt_day"]]

# convert to datetime
df_var["date"] = pd.to_datetime(df["date"], format = "%d.%m.%Y")

# set date as index
df_var.set_index("date", inplace = True)

# resample/groupby by date and convert to frame the total daily sales
df_var = df_var.resample("M")["item_cnt_day"].sum().to_frame()

# calculate the intra week variation between total sales
df_var["Variation"] = df_var["item_cnt_day"].diff()/df_var["item_cnt_day"].shift(1)

df_var.head()

In [ ]:
# separate x and y
y_sales = df_var["item_cnt_day"]
y_variation = df_var["Variation"]

# instanciate the figure
fig = plt.figure(figsize = (15, 10))
ax = fig.add_subplot(111)

# plot the total sales
plot1 = ax.plot(y_sales, label = "Total monthly sales", color = "blue", alpha = 0.5)

# create a secondary axis and plot the variation data
ax_bis = ax.twinx()
plot2 = ax_bis.plot(y_variation, label = "Intra - month variation of sales", color = "red", alpha = 0.5)

# create a common legend for both plots
lns = plot1 + plot2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc = "upper left")

# add a custom title to the plot
ax.set_title("Total monthly sales and variation");

<a id ="question_1"></a>
# Question 1: Create a plot with the moving average of total sales (7 days) and the variation on the second axis.
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# start with the regular df
df_for_question_1 = create_df()

In [ ]:
# select the columns of interest
df_1 = df_for_question_1[['date', 'item_cnt_day']]

# convert to datetime
df_1['date'] = pd.to_datetime(df_for_question_1['date'], format = '%d.%m.%Y')

# set date as index
df_1.set_index('date', inplace = True)

# resample/groupby by date and convert to frame the total daily sales
df_1 = df_1.resample('D')['item_cnt_day'].sum().to_frame()

# calculate moving average for 7 days
    
df_1['MA7D'] = df_1['item_cnt_day'].rolling(window=7).mean()

# calculate the variation of the moving average for 7 days
df_1['Variation_MA7D'] = df_1['MA7D'].diff()/df_1['MA7D'].shift(1)

df_1.head()

In [ ]:
# separate x and y
y_moving_average = df_1['MA7D'] # --> moving average en un eje
y_variation = df_1['Variation_MA7D'] # --> variación en otro eje

# instanciate the figure
fig = plt.figure(figsize = (15, 10))
ax = fig.add_subplot(111) # --> 111 un único subplot

# plot the total sales
plot1 = ax.plot(y_moving_average, label = "MA7D", color = "blue", alpha = 0.5)

# create a secondary axis and plot the variation data
ax_bis = ax.twinx() #--> twinx creo 2nd copia
plot2 = ax_bis.plot(y_variation, label = 'Variation_MA7D', color = "red", alpha = 0.5)

# create a common legend for both plots (hay que iterar por cada plot para ponerle la leyenda)
lns = plot1 + plot2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc = "upper left")

# add a custom title to the plot
ax.set_title('Moving average of total sales (7 days) and variation')

<a id = "calendar_heatmap"></a>
# Calendar heatmap
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# calendar heatmaps are really useful to see the overall activity for a certain period of time per day and per month.
# let's build one using python.
# we will be using the calmap package for this, because it makes it extremenly easy to plot this data
# select the columns
df_calendar = df[["date", "item_cnt_day"]]

# set date as index and resample
df_calendar.set_index("date", inplace = True)
# notice that this time, we don't convert it to_frame()
# df_calendar is a pandas series
# THIS IS IMPORTANT since calmap expects a series
# with a datetime index and the values to plot
df_calendar = df_calendar.resample("D")["item_cnt_day"].sum()

# ----------------------------------------------------------------------------------------------------
# plot the data using calmap
calmap.calendarplot(df_calendar, # pass the series
                    fig_kws = {'figsize': (16,10)}, 
                    yearlabel_kws = {'color':'black', 'fontsize':14}, 
                    subplot_kws = {'title':'Total sales per year'}
                   );

<a id = "corr_plots_daily"></a>
# Timeseries autocorrelation and partial autocorrelation plots: daily sales
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# This plot are fundamental in timeseries analysis.
# Basically here we compare the a series again itself but with some lags.
# These are plots that graphically summarize the strength of a relationship with an observation in a time series with observations at prior time steps.

# More info: 
# https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/

# ----------------------------------------------------------------------------------------------------
# instanciate the figure
fig, (ax1, ax2) = plt.subplots(1, 2,figsize = (16,6), dpi = 80)

# ----------------------------------------------------------------------------------------------------
# plot the data using the built in plots from the stats module

# The AutoCorrelation plot: compares a value v with the value v but n times in the past.
plot_acf(df.set_index("date").resample("D")["item_cnt_day"].sum(), ax = ax1, lags = 14)

# The Parcial AutoCorrelation plot: partial autocorrelation at lag k is the correlation that results after removing the effect of any correlations due to the terms at shorter lags.
plot_pacf(df.set_index("date").resample("D")["item_cnt_day"].sum(), ax = ax2, lags = 14);

<a id = "autocorrelation_calculation"></a>
# Manually calculate the Partial Autocorrelation
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# This code snippets show you have to calculate the Partial Autocorrelation
# Partial Autocorrelation can be very counter intuitive since in some of our steps we are fitting a linear model
# to predict the values of t - 2 using t - 1
# Wait, what? Why we use values from yesterday to predict values before yesterday?
# Basically because we assume that our timeseries is auto regressive. This means that the data at point t captures
# all the variance/information from all the previuos data points.
# This way, t - 1, must have captured all the variance from previous points, thus t - 2, and so t - 1 becomes
# a good predictor for values from t - 2.

In [ ]:
# create a dataframe with total sales per day (all shops and all items)
df_total_sales = df.set_index("date").resample("D")["item_cnt_day"].sum().to_frame()

# rename the column item_cnt_day to total_sales
df_total_sales.columns = ["total_sales"]


In [ ]:
df_total_sales.head()

In [ ]:

# create a few features that we need in order to calculate the parcial autocorrelation
df_total_sales["T-1"] = df_total_sales["total_sales"].shift(1)
df_total_sales["T-2"] = df_total_sales["total_sales"].shift(2)

# we have a few nan for the first 2 rows so we must drop them
print(df_total_sales.shape)
df_total_sales.dropna(axis = "rows", inplace = True)
print(df_total_sales.shape)

In [ ]:
# instanciate the Linear model
model = LinearRegression()

# separate X and y
X = df_total_sales[["T-1"]]
y = df_total_sales["total_sales"]

# fit and predict with the model
model.fit(X, y)
predictions = model.predict(X)

# save our predictions to the total_sales df
df_total_sales["total_sales_from_T-1"] = predictions

In [ ]:
df_total_sales.head()

In [ ]:
# instanciate the Linear model
model = LinearRegression()

# separate X and y
X = df_total_sales[["T-1"]]
y = df_total_sales["T-2"]

# fit and predict with the model
model.fit(X, y)
predictions = model.predict(X)

# save our predictions to the total_sales df
df_total_sales["T-2_from_T-1"] = predictions

In [ ]:
df_total_sales.head()

In [ ]:
# calculate the residual
# this means: total_sales - total_sales_from_T-1
# and: T-2 - "T-2_from_T-1"
df_total_sales["Residual_total_sales_T-1"] = df_total_sales["total_sales"] - df_total_sales["total_sales_from_T-1"]

# this step is very important based on the asumptions we have about many of the timeseries
# for more information I recommend this read
# https://towardsdatascience.com/understanding-partial-auto-correlation-fa39271146ac
df_total_sales["Residual_T-2_T-1"] = df_total_sales["T-2"] - df_total_sales["T-2_from_T-1"]

In [ ]:
df_total_sales.head()

In [ ]:
# calculathe the parcial autocorrelation using manual method
manual_pacf = df_total_sales.corr(method = "pearson")["Residual_total_sales_T-1"]["Residual_T-2_T-1"]
print("Manual parcial autocorrelation method {}".format(round(manual_pacf, 5)))

# calculate the parcial autocorrelation using statsmodel package
stats_pacf = pacf(df_total_sales['total_sales'], nlags = 2)[2]
print("Parcial autocorrelation method using stats package {}".format(round(stats_pacf, 5)))

<a id = "decomp_weekly"></a>
# Timeseries decomposition plots: weekly sales
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
df_timeindex = df.set_index("date").resample("W")["item_cnt_day"].sum().to_frame()

# decompose the series using stats module
# results in this case is a special class 
# whose attributes we can acess
result = seasonal_decompose(df_timeindex["item_cnt_day"])

# ----------------------------------------------------------------------------------------------------
# instanciate the figure
# make the subplots share teh x axis
fig, axes = plt.subplots(ncols = 1, nrows = 4, sharex = True, figsize = (12,10))

# ----------------------------------------------------------------------------------------------------
# plot the data
# using this cool thread:
# https://stackoverflow.com/questions/45184055/how-to-plot-multiple-seasonal-decompose-plots-in-one-figure
# This allows us to have more control over the plots

# plot the original data
result.observed.plot(ax = axes[0], legend = False)
axes[0].set_ylabel('Observed')
axes[0].set_title("Decomposition of a series")

# plot the trend
result.trend.plot(ax = axes[1], legend = False)
axes[1].set_ylabel('Trend')

# plot the seasonal part
result.seasonal.plot(ax = axes[2], legend = False)
axes[2].set_ylabel('Seasonal')

# plot the residual
result.resid.plot(ax = axes[3], legend = False)
axes[3].set_ylabel('Residual')

# ----------------------------------------------------------------------------------------------------
# prettify the plot

# get the xticks and the xticks labels
xtick_location = df_timeindex.index.tolist()

# set x_ticks
ax.set_xticks(xtick_location);

<a id = "question_2"></a>
# Question 2: Create a decomposition plot for a city of weekly sales
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# start with the regular df
df_for_question_2 = create_df()

In [ ]:
# select the city
city = 'Адыгея'
df_2 = df_for_question_2[df_for_question_2["city"] == city][["date","item_cnt_day"]]

# convert to datetime
df_2['date'] = pd.to_datetime(df['date'], format = '%d.%m.%Y')
df_2 = df_2.set_index("date").resample("W")["item_cnt_day"].sum().to_frame()

# decompose the series using stats module
result_df_2 = seasonal_decompose(df_2["item_cnt_day"])

# instanciate the figure
# make the subplots share teh x axis
fig, axes = plt.subplots(ncols = 1, nrows = 4, sharex = True, figsize = (12,10))

# ----------------------------------------------------------------------------------------------------
# plot the data

# plot the original data
result_df_2.observed.plot(ax = axes[0], legend = False)
axes[0].set_ylabel('Observed')
axes[0].set_title("Decomposition of 'Адыгея' serie of weekly sales")

# plot the trend
result_df_2.trend.plot(ax = axes[1], legend = False)
axes[1].set_ylabel('Trend')

# plot the seasonal part
result_df_2.seasonal.plot(ax = axes[2], legend = False)
axes[2].set_ylabel('Seasonal')

# plot the residual
result_df_2.resid.plot(ax = axes[3], legend = False)
axes[3].set_ylabel('Residual')

# ----------------------------------------------------------------------------------------------------
# prettify the plot

# get the xticks and the xticks labels
xtick_location = df_2.index.tolist()

# set x_ticks
ax.set_xticks(xtick_location);

<a id = "viz_cities"></a>
# Visualizing the most important cities
[Go back to the Table of Contents](#table_of_contents)

Treemaps are a very useful and visual tools to see different categories and their overall importance in a dataset.
Also, they are very cool and easy to make using Python and squarify.

In [ ]:
# prepare the data

# extract each year using dt.year
df["YEAR"] = df["date"].dt.year

# create a smaller df for year 2013
short_df = df[df["YEAR"] == 2013][["item_cnt_day", "city"]]

# groupby by city and sum all the sales
short_df = short_df.groupby("city")["item_cnt_day"].sum().to_frame()

# sort the values in the smaller df inplace
short_df.sort_values("item_cnt_day", ascending = False, inplace = True)

# we will do the same plot as before but without custom colors
# Moscow is a big outlier so it pales the rest of the cities

short_df = df[df["YEAR"] == 2014][["item_cnt_day", "city"]]
short_df = short_df.groupby("city")["item_cnt_day"].sum().to_frame()
short_df.sort_values("item_cnt_day", ascending = False, inplace = True)

my_values = short_df["item_cnt_day"]
my_pct = short_df["item_cnt_day"]/short_df["item_cnt_day"].sum()
labels = ['{} - Sales :{}k \n {}% of total'.format(city, sales/1000, round(pct, 2)*100) for city, sales, pct in zip(short_df.index, my_values, my_pct)]

plt.figure(figsize = (30, 8))
squarify.plot(sizes = my_values, label = labels,  alpha = 0.8)
plt.title("Sales by city and their % over total sales in 2014",fontsize = 23, fontweight = "bold")

plt.axis('off')
plt.tight_layout()

In [ ]:
df[["city", "city_id"]].drop_duplicates()

In [ ]:
# treemaps are very useful to see the difference and the weights of categories
# but they don't give us that much of information about the distribution of each category
# let's use boxplot to see the distribution of Moscow city

# we can see huge outliers for Moscow city.
plt.figure(figsize = (10, 10))
sns.boxplot(x = "city",
            y = "item_cnt_day", 
            data = df[(df["YEAR"] == 2013) & (df["city_id"] == 13)]
           );

<a id = "question_3"></a>
# Question 3: Create a treemap plot for item_category and the total combined sales

<span style="color:red">If the % of a category over total is less 1%, don't put any label!!!</span>

[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# start with the regular df
df_for_question_3 = create_df()

In [ ]:
# prepare the data
# create a smaller df
df_3 = df_for_question_3[["item_cnt_day", "item_category_name"]]

# groupby by category and sum all the sales
df_3 = df_3.groupby( "item_category_name")["item_cnt_day"].sum().to_frame()

# sort the values in the smaller df inplace
df_3.sort_values("item_cnt_day", ascending = False, inplace = True)

# get the x and y values
my_values = df_3["item_cnt_day"]
my_pct = df_3["item_cnt_day"]/df_3["item_cnt_day"].sum()

# create custom labels for each city with their total sales and overall contribution

def change(categoryx, salesx, pctx):
    text = ""
    if pctx > 0.009: text ='{} \n Sales :{}k \n {}% of total'.format(categoryx, salesx/1000, round(pctx, 2)*100)
    return text

labels = [change(category, sales, pct) for category, sales, pct in zip(df_3.index, my_values, my_pct)]


# create a color palette, mapped to the previous values
cmap = matplotlib.cm.Blues

# we want to normalize our values, otherwise a city will have the darkest collor and all the others will pale
mini = min(my_values)
maxi= np.percentile(my_values, q = 99)
norm = matplotlib.colors.Normalize(vmin = mini, vmax = maxi)
colors = [cmap(norm(value)) for value in my_values]

# instanciate the figure
plt.figure(figsize = (30, 10))

# we can pass colors but Moscow is way too big and most of the cities are pale blue
squarify.plot(sizes = my_values, label = labels,  alpha = 0.8)

# squarify.plot(sizes = my_values, label = labels,  alpha = 0.8)
# Remove our axes, set a title and display the plot
plt.title("Sales by category and their % over total sales", fontsize = 23, fontweight = "bold")
plt.axis('off')
plt.tight_layout()

<a id = "viz_null_values"></a>
# Visualizing nulls values
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# This plot will help us visualize the missing values for each datetime and item_id
# This is the most granular plots possible, since we will be seeing individual sales by day and item_id
# This plot can be very consufing, but the main point is to show all the "missing values" we have
# We have seen previously in our EDA, that when we groupby and resamples our sales, we might think
# that we don't have any missing values. But its not true, we only have the reported sales
# This means that, if we have a shop or item_id that only had 3 sales per year, when we resample
# our df by day, pandas will generate additional days with null sales.
# those null sales is what we want to plot here
# the values are ordered from less nulls to more nulls

gb_df_ = df.pivot_table(index = ["date"], columns = ['item_id'], values = "item_cnt_day", aggfunc = sum).isnull()
order_of_columns = list(gb_df_.sum().sort_values().index)
gb_df_ = gb_df_[order_of_columns]
plt.figure(figsize = (20, 10))
plot = sns.heatmap(gb_df_, cbar = True, cmap = "inferno")
plot.set_title("Null sales by item_id and day");

In [ ]:
gc.collect()
del gb_df_

In [ ]:
df[["shop_name","shop_id"]].drop_duplicates()

In [ ]:
# this will allow us to see a all the columns of the df
pd.options.display.max_columns = 999

In [ ]:
# create a smaller df
short_df = df[["date", "item_cnt_day", "shop_name"]]
# set the date to be the index (to resample later)
short_df.set_index("date", inplace = True)
# groupby by shop_name
gb = short_df.groupby("shop_name")
# resample the df by month sales (resample = groupby by months in timeseries)
gbr = gb.resample("M")["item_cnt_day"].sum()
# unstack the gbr to have columns name
gbr = gbr.unstack(level = -1).T
# sort the values, from no nulls to more null values
order_of_columns = list(gbr.isnull().sum().sort_values().index)
# change the order of the df
gbr = gbr[order_of_columns]

In [ ]:
# let's plot the null values for each shop
plt.figure(figsize=(20, 10))
# this lines gbr.unstack(level = -1).T.isnull()*1
# converts any null to 1 and the rest will be 0
sns.heatmap(gbr.isnull()*1, cmap = "inferno", cbar = True).set_title("Null values by shop and Month");

In [ ]:
# create a smaller df
short_df = df[["date", "item_cnt_day", "item_category_name"]]

# set the date to be the index (to resample later)
short_df.set_index("date", inplace = True)

# groupby by shop_name
gb = short_df.groupby("item_category_name")

# resample the df by month sales (resample = groupby by months in timeseries)
gbr = gb.resample("M")["item_cnt_day"].sum()

# unstack the gbr to have columns name
gbr = gbr.unstack(level = -1).T

# sort the values, from no nulls to more null values
order_of_columns = list(gbr.isnull().sum().sort_values().index)

# change the order of the df
gbr = gbr[order_of_columns]

In [ ]:
# let's plot the null values for each shop
plt.figure(figsize=(20, 10))

# this lines gbr.unstack(level = -1).T.isnull()*1
# converts any null to 1 and the rest will be 0
sns.heatmap(gbr.isnull()*1, cmap = "inferno", cbar = True).set_title("Null values by item category and Month");

<a id = "viz_outliers"></a>
# Visualization of outliers
[Go back to the Table of Contents](#table_of_contents)

In [ ]:
# let's look at outliers for item sales
# We will use boxplots because they are very useful to see the distribution of values
plt.figure(figsize = (10,4))
sns.boxplot(x = df["item_cnt_day"]);

In [ ]:
# let's look at outliers for item price
plt.figure(figsize = (10,4))
plt.xlim(df["item_price"].min(), df["item_price"].max()*1.1)
sns.boxplot(x = df["item_price"]);

In [ ]:
# joint plot is another very convenient way to plot the relationship between 2 variables
# but because we have huge outliers, we don't see them 
# https://seaborn.pydata.org/generated/seaborn.jointplot.html
plt.figure(figsize = (10,4))
sns.jointplot(x = "item_price", y = "item_cnt_day", data = df);

In [ ]:
# let's filter the outliers and make the same joint plot
df = df[(df["item_price"] < np.percentile(df["item_price"], q = 99)) & (df["item_cnt_day"] >= 0) & (df["item_cnt_day"] < np.percentile(df["item_cnt_day"], q = 99))]

In [ ]:
# we have removed the outliers and now 
plt.figure(figsize = (10, 10))
sns.jointplot(x = "item_price", y = "item_cnt_day", data = df);

<a id = "conclusion"></a>
# Conclusion
[Go back to the Table of Contents](#table_of_contents)

After taking a look at the sales data, here are some conclusion we can extract:

1. We see that the total sales decrease over time. This is very important because, we have to create features for our model that catch this trend.

2. We have seen that the sales present huge spikes in Christmas season. Datetime features can help a lot our model.

3. Data has a lot of missing values and we have not found a specific or category affected by this. More likely it's just the nature of the data.

4. Top 3 cities capture more than 50% of total sales. City based features can be very helpful for the model.

5. The top 3 categories represent more than 40% of total sales: they are Movies, PC Games and Music.

6. Data presents outliers at the sales and price level. Before generating features or training a model, data must be cleaned properly.

7. We have seen thanks to our calendar plots that we a small increase in sales on the weekends. We do see however bigger sales on 14 of February or 9 of May (holidays).